In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os

# Define the encoder
def create_encoder(input_shape):
    encoder_input = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(encoder_input)
    x = tf.keras.layers.MaxPooling3D((2, 2, 2), padding='same')(x)
    encoder_output = tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
    return tf.keras.Model(encoder_input, encoder_output)

# Define the decoder
def create_decoder(encoded_shape):
    decoder_input = tf.keras.Input(shape=encoded_shape)
    x = tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(decoder_input)
    x = tf.keras.layers.UpSampling3D((2, 2, 2))(x)
    decoder_output = tf.keras.layers.Conv3D(3, (3, 3, 3), activation='sigmoid', padding='same')(x)
    return tf.keras.Model(decoder_input, decoder_output)

# Combine encoder and decoder
input_shape = (16, 128, 128, 3)  # Example input shape
encoder = create_encoder(input_shape)
encoded_shape = encoder.output_shape[1:]
decoder = create_decoder(encoded_shape)

autoencoder_input = tf.keras.Input(shape=input_shape)
encoded = encoder(autoencoder_input)
decoded = decoder(encoded)

autoencoder = tf.keras.Model(autoencoder_input, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Summary of the model
autoencoder.summary()

# Function to load and preprocess videos
def load_videos(folder, frame_size, frame_count):
    frames = []
    for filename in os.listdir(folder):
        if filename.endswith('.avi'):
            cap = cv2.VideoCapture(os.path.join(folder, filename))
            video_frames = []
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, frame_size)
                video_frames.append(frame)
                if len(video_frames) == frame_count:
                    frames.append(video_frames)
                    video_frames = []
            cap.release()
    frames = np.array(frames)
    frames = frames / 255.0  # Normalize
    return frames

# Load and preprocess videos
frame_size = (128, 128)
frame_count = 16
video_frames = load_videos('h1/brush_hair', frame_size, frame_count)

# Train the autoencoder
autoencoder.fit(video_frames, video_frames, epochs=10, batch_size=1)

# Encode and decode video frames
encoded_frames = encoder.predict(video_frames)
decoded_frames = decoder.predict(encoded_frames)

# Function to save video
def save_video(filename, frames, frame_size):
    height, width = frame_size
    video = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))
    for frame_set in frames:
        for frame in frame_set:
            frame = (frame * 255).astype(np.uint8)
            video.write(frame)
    video.release()

# Save reconstructed video
save_video('reconstructed_cat.mp4', decoded_frames, frame_size)

print("Video processing complete.")


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 16, 128, 128,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional (Functional)         │ (None, 8, 64, 64, 128) │       226,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_1 (Functional)       │ (None, 16, 128, 128,   │       452,867 │
│                                 │ 3)                     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 679,427 (2.59 MB)

 Trainable params: 679,427 (2.59 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 455s 5s/step - loss: 0.0477
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 516s 5s/step - loss: 0.0135
Epoch 3/10
11/97 ━━━━━━━━━━━━━━━━━━━━ 9:57 7s/step - loss: 0.0040 

In [ ]:
autoencoder.save('g.h5')
encoder.save('e.h5')
decoder.save('d.h5')

In [ ]:
# Encode and decode video frames
encoded_frames = encoder.predict(video_frames)
decoded_frames = decoder.predict(encoded_frames)

# Function to save video
def save_video(filename, frames, frame_size):
    height, width = frame_size
    video = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))
    for frame_set in frames:
        for frame in frame_set:
            frame = (frame * 255).astype(np.uint8)
            video.write(frame)
    video.release()

# Save reconstructed video
save_video('reconstructed_cat.mp4', decoded_frames, frame_size)

print("Video processing complete.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 81s 13s/step
Video processing complete.
